<a href="https://colab.research.google.com/github/kyrajeep/Deep-Learning-and-Other/blob/master/hyperparameter_tuning_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Run hyperparameter tuning for a TensorFlow model

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/training/hyperparameter_tuning_tensorflow.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/training/hyperparameter_tuning_tensorflow.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/training/hyperparameter_tuning_tensorflow.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.9

## Overview

The values you select for your model’s hyperparameters can make all the difference. If you’re only trying to tune a handful of hyperparameters, you might be able to run experiments manually. But when you start juggling hyperparameters for your model’s architecture, the optimizer, and finding the best batch size and learning rate, automating these experiments at scale quickly becomes a necessity.

And it’s not just about tracking the results from all these trials. You also want a way to efficiently search the space of possible values so you don’t waste as much time trying out combinations that yield low accuracy scores.

Vertex AI Training includes a hyperparameter tuning service. A Vertex AI Hyperparameter tuning job will run multiple trials of your training code. On each trial, it will use different values for your chosen hyperparameters, set within limits you specify. By default, the service uses Bayesian optimization to search the space of possible hyperparameter values. This means that information from prior experiments is used to select the next set of values, making the search more efficient.

Learn more about [Vertex AI Hyperparameter Tuning](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview).

### Objective

In this tutorial, you learn how to run a Vertex AI Hyperparameter Tuning job for a TensorFlow model. While this example uses TensorFlow, you can also use this service for other ML frameworks.

This tutorial uses the following Google Cloud ML services and resources:

* Vertex AI Training
* Cloud Storage
* Artifact Registry


The steps performed include:

* Modify training application code for automated hyperparameter tuning.
* Containerize training application code.
* Configure and launch a hyperparameter tuning job with the Vertex AI Python SDK.

### Dataset

This sample uses the [Horses or Humans dataset](https://www.tensorflow.org/datasets/catalog/horses_or_humans) available through [TensorFlow datasets](https://www.tensorflow.org/datasets) to train a binary image classification model (horse or human).

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* Artifact Registry

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
[Cloud Storage pricing](https://cloud.google.com/storage/pricing),
and [Artifact Registry pricing](https://cloud.google.com/artifact-registry/pricing)
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages required to execute this notebook.

In [10]:
! pip3 install --upgrade --quiet google-cloud-aiplatform

In [11]:
!apt-get -qq install docker.io
!docker --version
! docker info
! which docker
! export PATH=$PATH:/usr/bin/docker


Docker version 24.0.5, build 24.0.5-0ubuntu1~22.04.1
Client:
 Version:    24.0.5
 Context:    default
 Debug Mode: false

Server:
ERROR: Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?
errors pretty printing info
/usr/bin/docker


In [12]:
#!pip3 install udocker


In [13]:
'''
older script for docker in colab but not working.
%%shell
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker info
docker network ls
docker pull hello-world
docker images
docker run hello-world
kill $(jobs -p)
'''

'\nolder script for docker in colab but not working.\n%%shell\nset -x\ndockerd -b none --iptables=0 -l warn &\nfor i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done\ndocker info\ndocker network ls\ndocker pull hello-world\ndocker images\ndocker run hello-world\nkill $(jobs -p)\n'

In [54]:
'''
# First let's update all the packages to the latest ones with the following command
! sudo apt update -qq

# Now we want to install some prerequisite packages which will let us use HTTPS over apt
!sudo apt install apt-transport-https ca-certificates curl software-properties-common -qq

# After that we will add the GPG key for the official Docker repository to the system
!curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -

# We will add the Docker repository to our APT sources
!sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu bionic stable"

# Next let's update the package database with our newly added Docker package repo
!sudo apt update -qq

# Finally lets install docker with the below command
!sudo apt install docker-ce
'''
# Lets check that docker is running
!docker start help

Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?
Error: failed to start containers: help


### Colab only: Uncomment the following cell to restart the kernel.

In [15]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [16]:
PROJECT_ID = "vision-412415"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


#### Set the region

**Optional**: Update the 'REGION' variable to specify the region that you want to use. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [27]:
REGION = "us-west4"  # @param {type: "string"}

#### UUID

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a uuid for each instance session, and append it onto the name of resources you create in this tutorial.

In [18]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### Authenticate your Google Cloud account

To authenticate your Google Cloud account, follow the instructions for your Jupyter environment:

**1. Vertex AI Workbench**
<br>You are already authenticated.

**2. Local JupyterLab instance**
<br>Uncomment and run the following code:

In [19]:
# ! gcloud auth login

**3. Colab**
<br>Uncomment and run the following code:

In [20]:
from google.colab import auth

auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [21]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [22]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Creating gs://your-bucket-name-vision-412415-unique/...
BadRequestException: 400 The specified location constraint is not valid.


### Import libraries

In [23]:
import os

import google.cloud.aiplatform as aiplatform
from google.cloud.aiplatform import hyperparameter_tuning as hpt

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [28]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

## Containerize training application code

Before you can run a hyperparameter tuning job, you must create a source code file (training script) and a Dockerfile.

The source code trains a model in the ML framework of your choice. In this example, you'll use TensorFlow to train a classification model.

The Dockerfile will include all the commands needed to run the image. It'll install all the libraries required by your training script, and set up the entry point for the training code.

You'll create a directory for storing your source code file and Dockerfile.

In [29]:
APPLICATION_DIR = "hptune"
TRAINER_DIR = f"{APPLICATION_DIR}/trainer"

In [30]:
!mkdir -p $APPLICATION_DIR
!mkdir -p $TRAINER_DIR

### Write the training script

In [31]:
%%writefile {TRAINER_DIR}/task.py

import tensorflow as tf
import tensorflow_datasets as tfds
import argparse
import hypertune

NUM_EPOCHS = 10


def get_args():
  '''Parses args. Must include all hyperparameters you want to tune.'''

  parser = argparse.ArgumentParser()
  parser.add_argument(
      '--learning_rate',
      required=True,
      type=float,
      help='learning rate')
  parser.add_argument(
      '--momentum',
      required=True,
      type=float,
      help='SGD momentum value')
  parser.add_argument(
      '--num_units',
      required=True,
      type=int,
      help='number of units in last hidden layer')
  args = parser.parse_args()
  return args


def preprocess_data(image, label):
  '''Resizes and scales images.'''

  image = tf.image.resize(image, (150,150))
  return tf.cast(image, tf.float32) / 255., label


def create_dataset():
  '''Loads Horses Or Humans dataset and preprocesses data.'''

  data, info = tfds.load(name='horses_or_humans', as_supervised=True, with_info=True)

  # Create train dataset
  train_data = data['train'].map(preprocess_data)
  train_data  = train_data.shuffle(1000)
  train_data  = train_data.batch(64)

  # Create validation dataset
  validation_data = data['test'].map(preprocess_data)
  validation_data  = validation_data.batch(64)

  return train_data, validation_data


def create_model(num_units, learning_rate, momentum):
  '''Defines and compiles model.'''

  inputs = tf.keras.Input(shape=(150, 150, 3))
  x = tf.keras.layers.Conv2D(16, (3, 3), activation='relu')(inputs)
  x = tf.keras.layers.MaxPooling2D((2, 2))(x)
  x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(x)
  x = tf.keras.layers.MaxPooling2D((2, 2))(x)
  x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
  x = tf.keras.layers.MaxPooling2D((2, 2))(x)
  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dense(num_units, activation='relu')(x)
  outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
  model = tf.keras.Model(inputs, outputs)
  model.compile(
      loss='binary_crossentropy',
      optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum),
      metrics=['accuracy'])
  return model


def main():
  args = get_args()
  train_data, validation_data = create_dataset()
  model = create_model(args.num_units, args.learning_rate, args.momentum)
  history = model.fit(train_data, epochs=NUM_EPOCHS, validation_data=validation_data)

  # DEFINE METRIC
  hp_metric = history.history['val_accuracy'][-1]

  hpt = hypertune.HyperTune()
  hpt.report_hyperparameter_tuning_metric(
      hyperparameter_metric_tag='accuracy',
      metric_value=hp_metric,
      global_step=NUM_EPOCHS)


if __name__ == "__main__":
    main()

Writing hptune/trainer/task.py


### Understanding the training script

Before you build the container, let's take a deeper look at the code. There are a few components that are specific to using the hyperparameter tuning service.

The script imports the **hypertune library**:

`import hypertune`

The **function `get_args()`** defines a command-line argument for each hyperparameter that you want to tune. In this example, the hyperparameters that will be tuned are the learning rate, the momentum value in the optimizer, and the number of units in the last hidden layer of the model. Feel free to experiment with others. The values passed in those arguments are then used to set the corresponding hyperparameter in the code.

```
def get_args():
  '''Parses args. Must include all hyperparameters you want to tune.'''

  parser = argparse.ArgumentParser()
  parser.add_argument(
      '--learning_rate',
      required=True,
      type=float,
      help='learning rate')
  parser.add_argument(
      '--momentum',
      required=True,
      type=float,
      help='SGD momentum value')
  parser.add_argument(
      '--num_units',
      required=True,
      type=int,
      help='number of units in last hidden layer')
  args = parser.parse_args()
  return args
```


At the end of the `main()` function, the `hypertune` library is used to **define the metric you want to optimize**. In TensorFlow, the keras `model.fit` method returns a `History` object. The `History.history` attribute is a record of training loss values and metrics values at successive epochs. If you pass validation data to `model.fit` the `History.history` attribute will include validation loss and metrics values as well.

```
  hp_metric = history.history['val_accuracy'][-1]

  hpt = hypertune.HyperTune()
  hpt.report_hyperparameter_tuning_metric(
      hyperparameter_metric_tag='accuracy',
      metric_value=hp_metric,
      global_step=NUM_EPOCHS)
 ```

For example, if you trained a model for three epochs with validation data and provided accuracy as a metric, the `History.history` attribute would look similar to the following dictionary.

```
{
 "accuracy": [
   0.7795261740684509,
   0.9471358060836792,
   0.9870933294296265
 ],
 "loss": [
   0.6340447664260864,
   0.16712145507335663,
   0.04546636343002319
 ],
 "val_accuracy": [
   0.3795261740684509,
   0.4471358060836792,
   0.4870933294296265
 ],
 "val_loss": [
   2.044623374938965,
   4.100203514099121,
   3.0728273391723633
 ]
```

If you want the hyperparameter tuning service to discover the values that maximize the model's validation accuracy, you define the metric as the last entry (or `NUM_EPOCS` - 1) of the `val_accuracy` list. Then, pass this metric to an instance of `HyperTune`. You can pick whatever string you like for the `hyperparameter_metric_tag`, but you need to use the string again later when you kick off the hyperparameter tuning job.

### Write Dockerfile

After writing your training code, you create a Dockerfile. In the Dockerfile, you include all the commands needed to run the image. It'll install all the necessary libraries, including the CloudML Hypertune library, and set up the entry point for the training code.

In [32]:
%%writefile {APPLICATION_DIR}/Dockerfile

FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-8

WORKDIR /

# Installs hypertune library
RUN pip install cloudml-hypertune

# Copies the trainer code to the Docker image.
COPY trainer /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

Writing hptune/Dockerfile


### Build the container

You'll store the Docker image in Artifact Registry. First, create a Docker repository in Artifact Registry

In [33]:
REPO_NAME='horses-app'

! gcloud services enable artifactregistry.googleapis.com

if os.getenv("IS_TESTING"):
    ! sudo apt-get update --yes && sudo apt-get --only-upgrade --yes install google-cloud-sdk-cloud-run-proxy google-cloud-sdk-harbourbridge google-cloud-sdk-cbt google-cloud-sdk-gke-gcloud-auth-plugin google-cloud-sdk-kpt google-cloud-sdk-local-extract google-cloud-sdk-minikube google-cloud-sdk-app-engine-java google-cloud-sdk-app-engine-go google-cloud-sdk-app-engine-python google-cloud-sdk-spanner-emulator google-cloud-sdk-bigtable-emulator google-cloud-sdk-nomos google-cloud-sdk-package-go-module google-cloud-sdk-firestore-emulator kubectl google-cloud-sdk-datastore-emulator google-cloud-sdk-app-engine-python-extras google-cloud-sdk-cloud-build-local google-cloud-sdk-kubectl-oidc google-cloud-sdk-anthos-auth google-cloud-sdk-app-engine-grpc google-cloud-sdk-pubsub-emulator google-cloud-sdk-datalab google-cloud-sdk-skaffold google-cloud-sdk google-cloud-sdk-terraform-tools google-cloud-sdk-config-connector
    ! gcloud components update --quiet

!gcloud artifacts repositories create $REPO_NAME --repository-format=docker \
--location=$REGION --description="Docker repository"

! gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

Create request issued for: [horses-app]
Created repository [horses-app].
Adding credentials for: us-west4-docker.pkg.dev
Docker configuration file updated.


In [56]:
# WARNING: `docker` not in system PATH.
#`docker` and `docker-credential-gcloud` need to be in the same PATH in order to work correctly together.
# gcloud's Docker credential helper can be configured but it will not work until this is corrected.
#https://stackoverflow.com/questions/49780218/docker-credential-gcloud-not-in-system-path
! echo $PATH
#!sudo usermod -aG docker $USER. need to have the accessibility to docker with another command
#!sudo systemctl status docker
! docker info
! export PATH=$PATH:/var/run/docker.sock
#! ls /tools/google-cloud-sdk/bin
#! ls /usr/local/bin
#! ls /usr/bin
#! ls /usr/sbin
#! ln -s /tools/google-cloud-sdk/bin/ /usr/sbin/

/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down
Client:
 Version:    24.0.5
 Context:    default
 Debug Mode: false

Server:
ERROR: Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?
errors pretty printing info


In [41]:
! gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet


{
  "credHelpers": {
    "us-west4-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-west4-docker.pkg.dev
gcloud credential helpers already registered correctly.


Define a variable with the URI of your Docker image in Artifact Registry:

In [42]:
IMAGE_URI = (
    f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPO_NAME}/horse_human_hptune:latest"
)

Then, build the container by running the following:

In [43]:
cd $APPLICATION_DIR

[Errno 2] No such file or directory: 'hptune'
/content/hptune


In [49]:
#! docker start --help
#! systemctl start docker
#! dockered
#!sudo service docker start
#!gpasswd -a $USER docker
! cd /var/run
! ls

Dockerfile  trainer


In [50]:
!docker build ./ -t $IMAGE_URI

DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            Install the buildx component to build images with BuildKit:
            https://docs.docker.com/go/buildx/

Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?


Finally, push it to Artifact Registry

In [39]:
!docker push $IMAGE_URI

Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?


In [25]:

!gcloud compute accelerator-types list \
   --project={PROJECT_ID}

NAME                   ZONE                       DESCRIPTION
ct5l                   us-central1-a              ct5l
ct5lp                  us-central1-a              ct5lp
nvidia-a100-80gb       us-central1-a              NVIDIA A100 80GB
nvidia-h100-80gb       us-central1-a              NVIDIA H100 80GB
nvidia-l4              us-central1-a              NVIDIA L4
nvidia-l4-vws          us-central1-a              NVIDIA L4 Virtual Workstation
nvidia-tesla-a100      us-central1-a              NVIDIA A100 40GB
nvidia-tesla-k80       us-central1-a              NVIDIA Tesla K80
nvidia-tesla-p4        us-central1-a              NVIDIA Tesla P4
nvidia-tesla-p4-vws    us-central1-a              NVIDIA Tesla P4 Virtual Workstation
nvidia-tesla-t4        us-central1-a              NVIDIA T4
nvidia-tesla-t4-vws    us-central1-a              NVIDIA Tesla T4 Virtual Workstation
nvidia-tesla-v100      us-central1-a              NVIDIA V100
nvidia-l4              us-central1-b              NVIDIA L4

## Configure a hyperparameter tuning job

Now that your training application code is containerized, it's time to specify and run the hyperparameter tuning job.

To launch the hyperparameter tuning job, you need to first define the `worker_pool_specs`, which specifies the machine type and Docker image. The following spec defines one `n1-standard-4` machine with one `NVIDIA Tesla T4` GPU as the accelerator.

In [ ]:
# The spec of the worker pools including machine type and Docker image
# Be sure to replace PROJECT_ID in the `image_uri` with your project.

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": "nvidia-tesla-p4",
            "accelerator_type": "NVIDIA_TESLA_P4",
            "accelerator_count": 1,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPO_NAME}/horse_human_hptune:latest"
        },
    }
]

### Define parameter spec

Next, define the `parameter_spec`, which is a dictionary specifying the parameters you want to optimize. The **dictionary key** is the string you assigned to the command line argument for each hyperparameter, and the **dictionary value** is the parameter specification.

For each hyperparameter, you need to define the `Type` as well as the bounds for the values that the tuning service will try. Hyperparameters can be of type `Double`, `Integer`, `Categorical`, or `Discrete`. If you select the type `Double` or `Integer`, you need to provide a minimum and maximum value. And if you select `Categorical` or `Discrete` you need to provide the values. For the `Double` and `Integer` types, you also need to provide the scaling value. Learn more about [Using an Appropriate Scale](https://www.youtube.com/watch?v=cSoK_6Rkbfg).

In [ ]:
# Dictionary representing parameters to optimize.
# The dictionary key is the parameter_id, which is passed into your training
# job as a command line argument,
# And the dictionary value is the parameter specification of the metric.
parameter_spec = {
    "learning_rate": hpt.DoubleParameterSpec(min=0.001, max=1, scale="log"),
    "momentum": hpt.DoubleParameterSpec(min=0, max=1, scale="linear"),
    "num_units": hpt.DiscreteParameterSpec(values=[64, 128, 512], scale=None),
}

The final spec to define is `metric_spec`, which is a dictionary representing the metric to optimize. The dictionary key is the `hyperparameter_metric_tag` that you set in your training application code, and the value is the optimization goal.

In [ ]:
# Dictionary representing metrics to optimize.
# The dictionary key is the metric_id, which is reported by your training job,
# And the dictionary value is the optimization goal of the metric.
metric_spec = {"accuracy": "maximize"}

Once the specs are defined, you create a `CustomJob`, which is the common spec that will be used to run your job on each of the hyperparameter tuning trials.

In [ ]:
my_custom_job = aiplatform.CustomJob(
    display_name="horses-humans-sdk-job",
    worker_pool_specs=worker_pool_specs,
    staging_bucket=BUCKET_URI,
)

Then, create and run `HyperparameterTuningJob`.

In [ ]:
hp_job = aiplatform.HyperparameterTuningJob(
    display_name="horses-humans-sdk-job",
    custom_job=my_custom_job,
    metric_spec=metric_spec,
    parameter_spec=parameter_spec,
    max_trial_count=10,
    parallel_trial_count=3,
)

hp_job.run()

There are a few arguments to note:


* **max_trial_count**: You need to put an upper bound on the number of trials the service will run. More trials generally leads to better results, but there will be a point of diminishing returns, after which additional trials have little or no effect on the metric you're trying to optimize. It is a best practice to start with a smaller number of trials and get a sense of how impactful your chosen hyperparameters are before scaling up.

* **parallel_trial_count**: If you use parallel trials, the service provisions multiple training processing clusters. Increasing the number of parallel trials reduces the amount of time the hyperparameter tuning job takes to run; however, it can reduce the effectiveness of the job overall. This is because the default tuning strategy uses results of previous trials to inform the assignment of values in subsequent trials.

* **search_algorithm**: You can set the search algorithm to grid, random, or default (None). If you do not specify an algorithm, as shown in this example, the default option applies Bayesian optimization to search the space of possible hyperparameter values and is the recommended algorithm. Learn more about [Hyperparameter tuning using Bayesian Optimization](https://cloud.google.com/blog/products/ai-machine-learning/hyperparameter-tuning-cloud-machine-learning-engine-using-bayesian-optimization)

## Examine results

Click on the generated link in the output to see your run in the Cloud Console. When the job completes, you will see the results of the tuning trials.

You can sort the results by the optimization metric and then set the hyperparamers in your training application code to the values from the trial with the highest accuracy.

![console_ui_results](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/training/tuning_results.png?raw=1)

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [ ]:
# Delete artifact registry repo
! gcloud artifacts repositories delete $REPO_NAME --location $REGION --quiet

delete_custom_job = True
delete_bucket = False

# Delete hptune job
if delete_custom_job:
    try:
        hp_job.delete()
    except Exception as e:
        print(e)

# Delete bucket
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI